# Process Tail Simulation Results

In [1]:
import os, pickle

results_dir = r"D:\Ergodicity Simulations\2025-10-09 Run - Loss Tail and Limits - $25M and $50M"

# sample_file = "Cap (25M) - Ded (100K) - LR (0.6) - Pol_Lim (25M) - X_Th_%le (0.0005) - X_Shape (1.0) - X_Scale (1.0) - 250K Sims - 25 Yrs.pkl"

# file_path = os.path.join(results_dir, sample_file)
# with open(file_path, "rb") as f:
#     loaded_data = pickle.load(f)

In [2]:
# loaded_data.ruin_probability

In [3]:
# loaded_data.growth_rates.mean()

In [4]:
import pickle
import re

from pathlib import Path
from PIL import Image

from ergodic_insurance.monte_carlo import SimulationResults
import numpy as np
from time import perf_counter
from tqdm.auto import tqdm

def _parse_number(text):
    text = text.strip().replace(",", "")
    m = re.fullmatch(r'([+-]?\d+(?:\.\d+)?)([KMB])?$', text, re.I)
    if not m:
        # fallback: plain int/float or leave as-is
        try:
            return int(text)
        except ValueError:
            try:
                return float(text)
            except ValueError:
                return text
    num = float(m.group(1))
    mult = {"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}.get((m.group(2) or "").upper(), 1)
    val = num * mult
    return int(val) if val.is_integer() else val


def parse_config_key(key: str) -> dict:
    parts = re.split(r"\s*-\s*", key.strip())
    out = {}
    for part in parts:
        if not part:
            continue

        # e.g. "Cap (100M)"
        m = re.match(r"^([A-Za-z_%]+)\s*\(\s*([^)]+)\s*\)$", part)
        if m:
            out[m.group(1)] = _parse_number(m.group(2))
            continue

        # e.g. "0K Sims" or "50 Yrs"
        m = re.match(r"^([+-]?\d+(?:\.\d+)?)\s*([KMB])?\s*([A-Za-z_]+)$", part)
        if m:
            value = _parse_number((m.group(1) or "") + (m.group(2) or ""))
            out[m.group(3)] = value
            continue

        # flags like "NOINS"
        if part.upper() == "NOINS":
            out["NOINS"] = True

    return out

results_dir = Path(results_dir)

sample_files = "*.pkl"

pkl_paths = sorted(results_dir.glob(sample_files))

qs = np.arange(0.01, 1.00, 0.01) # Growth Rate Quantiles

all_configurations = {}
if not pkl_paths:
    print(f"No pickle files found in {results_dir}.")
else:
    try:
        iterator = tqdm(pkl_paths, desc="Processing pickle files", unit="file")
    except Exception:
        iterator = pkl_paths  # fallback without progress bar

    start_time = perf_counter()
    for idx, path in enumerate(iterator, 1):
        if idx > 1:
            elapsed = perf_counter() - start_time
            avg = elapsed / (idx - 1)
            remaining = avg * (len(pkl_paths) - (idx - 1))
            if hasattr(iterator, "set_postfix"):
                iterator.set_postfix(avg_s=f"{avg:.2f}", eta_s=f"{remaining:.1f}")
        try:
            with open(path, "rb") as f:
                one_config = pickle.load(f)
                growth_rate = one_config.growth_rates.mean()
                growth_rate_ci = {str(q): val for q, val in zip(qs, np.quantile(one_config.growth_rates, qs))}
                ror = one_config.ruin_probability
                all_configurations[path.stem] = {
                    "growth_rate": growth_rate,
                    "growth_rate_ci": growth_rate_ci,
                    "risk_of_ruin": ror
                    # "annual_losses": one_config.annual_losses
                }
        except Exception as e:
            print(f"Skipping {path.name}: {e}")
    print(f"Loaded {len(all_configurations)} pickle files into all_configurations.")

parsed_params_by_key = {k: parse_config_key(k) | all_configurations[k] for k in all_configurations.keys()}
# parsed_params_by_key

Processing pickle files:   0%|          | 0/342 [00:00<?, ?file/s]

Loaded 342 pickle files into all_configurations.


In [5]:
outfile = r"cache\parsed_params_by_key.pkl"
out_path = Path(outfile)
if out_path.parent and not out_path.parent.exists():
    out_path.parent.mkdir(parents=True, exist_ok=True)

with open(out_path, "wb") as f:
    pickle.dump(parsed_params_by_key, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Wrote {len(parsed_params_by_key)} configurations to {outfile}")

Wrote 342 configurations to cache\parsed_params_by_key.pkl


In [6]:
# import pickle

# infile = r"cache\parsed_params_by_key.pkl"

# with open(infile, 'rb') as f:
#     parsed_params_by_key = pickle.load(f)
# print(f"Loaded {len(parsed_params_by_key)} configurations to `parsed_params_by_key`")

In [7]:
# Replace the first failing comprehension with this safe version:
all_lims = sorted({sc['Pol_Lim'] for sc in parsed_params_by_key.values() if 'Pol_Lim' in sc})
all_lims

[25000000,
 50000000,
 75000000,
 100000000,
 150000000,
 250000000,
 350000000,
 500000000]

In [8]:
# Replace the first failing comprehension with this safe version:
all_keys = set().union(*(sc.keys() for sc in parsed_params_by_key.values()))
all_keys

{'Cap',
 'Ded',
 'LR',
 'NOINS',
 'Pol_Lim',
 'Sims',
 'X_Scale',
 'X_Shape',
 'X_Th_%le',
 'Yrs',
 'growth_rate',
 'growth_rate_ci',
 'risk_of_ruin'}

In [9]:
for key in all_keys:
    all_key_vals = sorted({str(sc[key]) for sc in parsed_params_by_key.values() if key in sc and type(sc[key]) not in (list, dict, set)})
    if all_key_vals != []:
        print(f"{key}: {all_key_vals}")
    all_key_vals = {type(sc[key]) for sc in parsed_params_by_key.values() if key in sc and type(sc[key]) in (list, dict, set)}
    if all_key_vals != set():
        print(f"{key}: {all_key_vals}")

X_Shape: ['0', '0.25', '0.5', '1', '1.5', '2', '2.5']
Sims: ['250000']
X_Th_%le: ['0.0001', '0.0005', '0.001', 'None']
risk_of_ruin: {<class 'dict'>}
X_Scale: ['1']
Yrs: ['50']
Cap: ['25000000', '50000000']
LR: ['0.6']
growth_rate_ci: {<class 'dict'>}
growth_rate: ['-0.00022857436085238419', '-0.0004914657667283132', '-0.0005489413834418659', '-0.0006144283931312216', '-0.0007488680177335948', '-0.0007932669721167476', '-0.00084273066879613', '-0.0011505915639255409', '-0.0012989714935845516', '-0.0014195588167289954', '-0.0021373325815433274', '-0.0023672637313773594', '-0.0024309830105667877', '-0.0024487611038656767', '-0.002480659918748632', '-0.0025421166390742172', '-0.002719388843804889', '-0.0028802205777773413', '-0.0032186547946578383', '-0.003345948024542558', '-0.003423411160741365', '-0.003678337430153145', '-0.0037178523932277956', '-0.003744173776939032', '-0.003950126011396696', '-0.003994024943184451', '-0.004302695096061416', '-0.004341131974803254', '-0.0043429190079

I have the following set of data as a list of dictionaries for 1029 configurations that I'd like to explore visually in Python:

```
'risk_of_ruin': {<class 'dict'>} # Risk of Ruin snapshots at 5-year intervales (5, 10, 15, 20, 25)
'Ded': ['100000'] # Deductible
'X_Scale': ['0.5', '1', '1.5', '2'] # Scale parameter for the Generalized Pareto Distribution (GPD) tail
'X_Th_%le': ['0.0001', '0.0005', '0.001', 'None'] # Threshold percentile for the Generalized Pareto Distribution (GPD) tail
'growth_rate': <class 'float'> # Mean Growth Rate at the end of the simulation
'growth_rate_ci': {<class 'dict'>} # Growth Rate quantiles from 0.01 to 0.99 in increments of 0.01 at the end of the simulation
'Pol_Lim': ['100000000', '200000000', '50000000', '500000000']
'X_Shape': ['0', '1', '1.5', '2', '2.5'] # Shape parameter for the Generalized Pareto Distribution (GPD) tail
'LR': ['0.3', '0.4', '0.5', '0.6', '0.7'] # Loss Ratio (Claims / Premiums), lower implies a higher premium charge for the same expected losses
'Yrs': ['25'] # Simulations were run for 25 years only
'Sims': ['100000'] # Each configuration was run for 100,000 simulations
# There are also configurations without insurance for which there is no 'Pol_Lim', no 'Ded', and no 'LR' parameters. These scenarios are marked with `'NOINS': True`
```

For the first set of plots, I'd like to create an Efficiency Frontier plot with the following parameters:
x-axix: 'risk_of_ruin'
y-axis: 'growth_rate'
Display this curve for each tail configuration in the following setup:
row: 'X_Th_%le'
column: 'X_Shape'
Plot different graphs for each collection of 'X_Scale', which are transformations on the loss distribution with '1' representing a close match, '0.5' representing 0.5 loss density compared to default tail, '2' representing twice the density of the default tail, and so on.
Color represents percentiles, using the pallette "cividis"
Encode different limits as shapes for the median/mean/quantile, so for example, a triangle in different colors represents the same limit (50M)
Shapes are as follows:
'NOINS': plus sign, "P" shape code
'50000000': triangle, "v" shape code
'100000000': square, "s" shape code
'200000000': pentagon, "p" shape code
'500000000': hexagon, "H" shape code

There should be 4 plots (one for each 'X_Scale') with DPI set to 300px that get saved to individual PNG files.


In [10]:
ins_only_vals = [val for val in parsed_params_by_key.values() if val.get('NOINS', False) is False]
len(ins_only_vals)

304

In [11]:
[c for c in ins_only_vals if c['X_Shape'] == 0]


[{'Cap': 25000000,
  'Ded': 100000,
  'LR': 0.6,
  'Pol_Lim': 100000000,
  'X_Th_%le': 'None',
  'X_Shape': 0,
  'X_Scale': 1,
  'Sims': 250000,
  'Yrs': 50,
  'growth_rate': np.float64(0.013144233279369233),
  'growth_rate_ci': {'0.01': np.float64(0.01275462723337114),
   '0.02': np.float64(0.012820132989436388),
   '0.03': np.float64(0.012859232984483241),
   '0.04': np.float64(0.012888167053461075),
   '0.05': np.float64(0.012911093095317483),
   '0.060000000000000005': np.float64(0.012930044792592526),
   '0.06999999999999999': np.float64(0.012946965647861361),
   '0.08': np.float64(0.012962017133831978),
   '0.09': np.float64(0.01297553451731801),
   '0.09999999999999999': np.float64(0.012987721711397171),
   '0.11': np.float64(0.012999445097520948),
   '0.12': np.float64(0.013010070659220219),
   '0.13': np.float64(0.013020103191956877),
   '0.14': np.float64(0.013029794748872519),
   '0.15000000000000002': np.float64(0.01303873024880886),
   '0.16': np.float64(0.0130473713949322

In [12]:
x_shapes = sorted(set(c['X_Shape'] for c in ins_only_vals))
x_shapes

[0, 0.25, 0.5, 1, 1.5, 2, 2.5]

In [13]:
[c for c in ins_only_vals if c['X_Shape'] == 0]

[{'Cap': 25000000,
  'Ded': 100000,
  'LR': 0.6,
  'Pol_Lim': 100000000,
  'X_Th_%le': 'None',
  'X_Shape': 0,
  'X_Scale': 1,
  'Sims': 250000,
  'Yrs': 50,
  'growth_rate': np.float64(0.013144233279369233),
  'growth_rate_ci': {'0.01': np.float64(0.01275462723337114),
   '0.02': np.float64(0.012820132989436388),
   '0.03': np.float64(0.012859232984483241),
   '0.04': np.float64(0.012888167053461075),
   '0.05': np.float64(0.012911093095317483),
   '0.060000000000000005': np.float64(0.012930044792592526),
   '0.06999999999999999': np.float64(0.012946965647861361),
   '0.08': np.float64(0.012962017133831978),
   '0.09': np.float64(0.01297553451731801),
   '0.09999999999999999': np.float64(0.012987721711397171),
   '0.11': np.float64(0.012999445097520948),
   '0.12': np.float64(0.013010070659220219),
   '0.13': np.float64(0.013020103191956877),
   '0.14': np.float64(0.013029794748872519),
   '0.15000000000000002': np.float64(0.01303873024880886),
   '0.16': np.float64(0.0130473713949322

In [14]:
lr = sorted(set(c['LR'] for c in ins_only_vals))
lr

[0.6]

## Plot Facets of Growth vs Limit

In [18]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from matplotlib.lines import Line2D

# Assuming your data is stored in a list called 'configurations'
# configurations = [{'risk_of_ruin': {...}, 'growth_rate': ..., 'X_Scale': ..., ...}, ...]


def create_efficiency_frontier_plots(configurations):
    """
    Create efficiency frontier plots for actuarial simulation results.
    
    FIXED VERSION: Includes comprehensive error handling for pcolormesh issues.

    Parameters:
    -----------
    configurations : list of dict
        List of configuration dictionaries with simulation results

    Output:
    -------
    Saves PNG files for each X_Scale with efficiency frontier plots.
    """

    # Get unique values for separate charts
    x_scales = sorted(set(c['X_Scale'] for c in configurations))
    lr = sorted(set(c['LR'] for c in configurations))
    caps = sorted(set(c['Cap'] for c in configurations))

    # Define percentiles for overlay
    highlight_percentiles = [0.50, 0.75, 0.90, 0.95, 0.99]
    legend_percentiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]

    legend_percentiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
    all_percentiles = np.arange(0.01, 1.00, 0.01).tolist()

    for cap in caps:
        # Create a plot for each X_Scale
        for scale in x_scales:
            for loss_ratio in lr:
                # CRITICAL FIX: Filter by cap, scale, AND loss_ratio to prevent duplicates
                scale_configs = [c for c in configurations 
                                if c['Cap'] == cap and c['X_Scale'] == scale and c['LR'] == loss_ratio]
                
                # Get unique values for faceting
                x_shapes = sorted(set(float(c['X_Shape']) for c in scale_configs))
                x_shapes = [s for s in x_shapes if s != 0] # Filter out 0
                x_shapes = [0.25, 0.5, 1.0, 1.5]
                # print(x_shapes)
                x_thresholds = sorted({(c['X_Th_%le'] if isinstance(c['X_Th_%le'], (float, np.floating)) else 0.0) for c in scale_configs})
                x_thresholds = [s for s in x_thresholds if s != 0] # Filter out 0

                all_gr_min, all_gr_max = [], []
                for val in ins_only_vals:
                    val_min, val_max = min(val['growth_rate_ci'].values()), max(val['growth_rate_ci'].values())
                    all_gr_min.append(val_min)
                    all_gr_max.append(val_max)

                y_min, y_max = min(all_gr_min), max(all_gr_max)

                # Create figure with subplots
                n_rows = len(x_thresholds)
                n_cols = len(x_shapes)

                fig, axes = plt.subplots(n_rows, n_cols,
                                        figsize=(5*n_cols, 4*n_rows),
                                        dpi=300,
                                        squeeze=False)

                # Set overall title
                cap_label = f"${cap/1_000_000:.0f}M"
                scale_label = f"{float(scale):.1f}" if scale != '1' else "Baseline"
                lr_pct = f"{loss_ratio*100:.0f}%"
                fig.suptitle(f'Efficiency Frontier (Init Cap: {cap_label}; Scale: {scale_label}; Loss Ratio: {lr_pct})',
                            fontsize=16, fontweight='bold', y=0.995)

                # Create colormap for percentiles
                cmap = plt.cm.viridis

                # Plot each facet
                for i, threshold in enumerate(x_thresholds):
                    for j, shape_param in enumerate(x_shapes):
                        ax = axes[i, j]

                        # Debug: Check what values we're comparing
                        if threshold == 0 or shape_param == 0:
                            print(
                                f"\n=== Debug for threshold={threshold}, shape_param={shape_param} ===")
                            print(
                                f"shape_param type: {type(shape_param)}, value: {repr(shape_param)}")
                            print(
                                f"threshold type: {type(threshold)}, value: {repr(threshold)}")

                            # Sample a few configs to see their actual values
                            for c in scale_configs[:3]:
                                print(
                                    f"  X_Shape: {repr(c['X_Shape'])} (type: {type(c['X_Shape'])})")
                                print(
                                    f"  X_Th_%le: {repr(c['X_Th_%le'])} (type: {type(c['X_Th_%le'])})")
                                print(
                                    f"  float(c['X_Shape']) == shape_param: {float(c['X_Shape']) == shape_param}")
                                print(
                                    f"  Threshold match: {c['X_Th_%le'] == threshold or (c['X_Th_%le'] is None and threshold == 0)}")
                                print()

                        # Filter for this facet
                        facet_configs = [c for c in scale_configs
                                        if (c['X_Th_%le'] == threshold or (c['X_Th_%le'] == 'None' and threshold == 0))
                                        and float(c['X_Shape']) == shape_param]

                        if (threshold == 0 or shape_param == 0):
                            print(len(facet_configs))

                        # Intermediate pass - Create smooth gradient using pcolormesh
                        # Organize data by percentile
                        percentile_data = {pct: [] for pct in all_percentiles}

                        for config in facet_configs:
                            # Calculate log of policy limit for x-axis
                            if config.get('NOINS', False):
                                continue  # Skip NOINS
                            else:
                                pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                    config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                                x_value = np.log10(float(pol_lim))
                            
                            # Collect percentile values
                            if 'growth_rate_ci' in config and config['growth_rate_ci']:
                                for pct in all_percentiles:
                                    pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                                    if pct_key in config['growth_rate_ci']:
                                        growth = config['growth_rate_ci'][pct_key]
                                        percentile_data[pct].append((x_value, growth, float(pol_lim)))

                        # Get unique x values (policy limits) and sort
                        unique_x_values = sorted(set(p[0] for pct_list in percentile_data.values() for p in pct_list))

                        if len(unique_x_values) > 1:
                            # Create interpolation grid with many points for smooth gradient
                            x_interp = np.linspace(min(unique_x_values), max(unique_x_values), 200)
                            
                            # For each percentile, interpolate growth rates across policy limits
                            from scipy.interpolate import interp1d
                            
                            y_grid = []
                            valid_percentiles = []
                            
                            for pct in sorted(all_percentiles):
                                if len(percentile_data[pct]) > 1:
                                    # Sort by x value
                                    sorted_data = sorted(percentile_data[pct], key=lambda p: p[0])
                                    x_data = np.array([p[0] for p in sorted_data])
                                    y_data = np.array([p[1] for p in sorted_data])
                                    
                                    # CRITICAL FIX: Remove non-finite values and use bounded extrapolation
                                    valid_mask = np.isfinite(x_data) & np.isfinite(y_data)
                                    if np.sum(valid_mask) >= 2:  # Need at least 2 points
                                        x_clean = x_data[valid_mask]
                                        y_clean = y_data[valid_mask]
                                        
                                        try:
                                            # Use bounded fill values instead of 'extrapolate' to prevent inf
                                            f = interp1d(x_clean, y_clean, kind='linear', 
                                                        bounds_error=False,
                                                        fill_value=(y_clean[0], y_clean[-1]))
                                            y_interp = f(x_interp)
                                            
                                            # Validate interpolation output
                                            if np.all(np.isfinite(y_interp)):
                                                y_grid.append(y_interp)
                                                valid_percentiles.append(pct)
                                            else:
                                                print(f"Warning: Non-finite interpolation at pct={pct}")
                                        except Exception as e:
                                            print(f"Interpolation failed at pct={pct}: {e}")
                            
                            if len(y_grid) > 1:
                                # Convert to 2D array: rows = percentiles, columns = x positions
                                y_grid = np.asarray(y_grid, dtype=np.float64)
                                
                                # CRITICAL FIX: Check for MaskedArray and validate data
                                if isinstance(y_grid, np.ma.MaskedArray):
                                    print(f"Warning: y_grid is MaskedArray, converting...")
                                    y_grid = y_grid.filled(np.nan)
                                
                                # Final validation: ensure all finite values
                                if not np.all(np.isfinite(y_grid)):
                                    print(f"Warning: Non-finite values in y_grid, cleaning...")
                                    # Replace inf with max/min finite
                                    finite_mask = np.isfinite(y_grid)
                                    if np.any(finite_mask):
                                        max_finite = np.max(y_grid[finite_mask])
                                        min_finite = np.min(y_grid[finite_mask])
                                        y_grid = np.where(np.isposinf(y_grid), max_finite, y_grid)
                                        y_grid = np.where(np.isneginf(y_grid), min_finite, y_grid)
                                        # Fill NaN with column means
                                        for col_idx in range(y_grid.shape[1]):
                                            col = y_grid[:, col_idx]
                                            if np.any(np.isnan(col)):
                                                col_mean = np.nanmean(col)
                                                y_grid[np.isnan(col), col_idx] = col_mean
                                
                                # Only proceed if we have valid data
                                if np.all(np.isfinite(y_grid)):
                                    # Create meshgrid for pcolormesh
                                    X, Y = np.meshgrid(x_interp, valid_percentiles)
                                    
                                    # Ensure meshgrids are regular arrays
                                    X = np.asarray(X, dtype=np.float64)
                                    Y = np.asarray(Y, dtype=np.float64)
                                    
                                    # Use pcolormesh with the colormap
                                    mesh = ax.pcolormesh(X, y_grid, Y, 
                                                        cmap=cmap, 
                                                        shading='gouraud',  # Smooth interpolation
                                                        vmin=0, vmax=1,
                                                        zorder=5)
                                else:
                                    print(f"Skipping pcolormesh - data still contains non-finite values")

                        # Second pass: Overlay specific percentiles with shapes
                        for config in facet_configs:
                            # Get risk of ruin at year 25
                            # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                            # Calculate log of policy limit for x-axis
                            if config.get('NOINS', False):
                                x_value = 0  # or np.nan if you want to exclude NOINS from plot
                            else:
                                pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                    config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                                x_value = np.log10(float(pol_lim))

                            # Plot mean in #cc0000
                            if 'growth_rate' in config:
                                growth_mean = config['growth_rate']
                                ax.scatter(x_value, growth_mean,
                                            marker='o',
                                            c='#cc0000',
                                            alpha=1.0,
                                            edgecolors='#cc0000',
                                            linewidths=1.5,
                                            zorder=10)


                        ### Draw lines connecting means across limits #######
                        # Collect (x, growth_rate, raw_limit) tuples to connect mean growth rates
                        mean_points = []
                        
                        for config in facet_configs:
                            # Skip NOINS for connecting line (only plot actual policy limits)
                            if config.get('NOINS', False):
                                continue
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            try:
                                x_value = np.log10(float(pol_lim))
                            except (TypeError, ValueError):
                                continue
                            if 'growth_rate' in config and config['growth_rate'] is not None:
                                mean_points.append((x_value, float(config['growth_rate']), float(pol_lim)))
                        
                        if len(mean_points) > 1:
                            # Sort by actual (non-log) policy limit to ensure correct ordering
                            sorted_points = sorted(mean_points, key=lambda p: p[2])
                            x_coords = [p[0] for p in sorted_points]
                            y_coords = [p[1] for p in sorted_points]
                            ax.plot(x_coords, y_coords,
                                    color='#cc0000',
                                    linewidth=2,
                                    alpha=1.0,
                                    zorder=9)

                        # Formatting
                        ax.set_xlabel('Policy Limit (log scale)', fontsize=10)
                        ax.set_ylabel('Growth Rate', fontsize=10)
                        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

                        # Set title for each subplot
                        thresh_label = f'{threshold*100:,.2f}%' if threshold != 'None' else 'No Tail'
                        ax.set_title(f'Tail Threshold: {thresh_label} ; Shape: {shape_param}',
                                    fontsize=9)

                        # Set axis limits
                        # ax.set_xlim(x_min, x_max)
                        ax.set_xlim(np.log10(25000000* 0.9),
                                    np.log10(500000000 * 1.1))
                        if (cap == 25_000_000):
                            ax.set_ylim(-0.15, 0.03)
                            # ax.set_ylim(y_min, y_max)
                        elif (cap == 50_000_000):
                            ax.set_ylim(-0.15, 0.03)
                            # ax.set_ylim(y_min, y_max)
                        else:
                            ax.set_ylim(y_min, y_max)

                        # Set x-ticks to show actual policy limit values
                        ax.set_xticks([np.log10(25e6),
                                        np.log10(50e6),
                                        np.log10(75e6),
                                        np.log10(100e6), 
                                        np.log10(150e6),
                                        np.log10(250e6),
                                        np.log10(350e6),
                                        np.log10(500e6)])
                        ax.set_xticklabels(['25M',
                                            '50M',
                                            '75M', 
                                            '100M', 
                                            '150M',
                                            '250M', 
                                            '350M',
                                            '500M'])

                # Color legend (percentiles + mean)
                color_elements = []
                for pct in legend_percentiles:
                    color_elements.append(mpatches.Patch(color=cmap(pct),
                                                        label=f'{int(pct*100)}th %ile',
                                                        edgecolor='black',
                                                        linewidth=0.5))
                color_elements.append(mpatches.Patch(color='#cc0000',
                                                    label='Mean',
                                                    edgecolor='#cc0000',
                                                    linewidth=0.5))

                fig.legend(handles=color_elements, loc='upper center',
                            bbox_to_anchor=(0.5, 0.0), title='Percentile',
                            frameon=False, fontsize=9, ncol=len(color_elements))

                plt.tight_layout(rect=[0, 0, 1, 0.99])

                # Save figure
                filename = f'cache/efficiency_frontier_cap_{cap_label}_scale_{str(scale).replace(".", "p")}_lr_{str(loss_ratio).replace(".", "p")}.png'
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                print(f'Saved: {filename}')
                plt.close()


create_efficiency_frontier_plots(ins_only_vals)

C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\3303078353.py:319: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color=cmap(pct),
C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\3303078353.py:323: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color='#cc0000',


Saved: cache/efficiency_frontier_cap_$25M_scale_1_lr_0p6.png


C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\3303078353.py:319: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color=cmap(pct),
C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\3303078353.py:323: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color='#cc0000',


Saved: cache/efficiency_frontier_cap_$50M_scale_1_lr_0p6.png


## Plot Individual Graphs of Growth vs Limit

In [16]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

from matplotlib.lines import Line2D
from scipy.interpolate import interp1d

# Assuming your data is sto#cc0000 in a list called 'configurations'
# configurations = [{'risk_of_ruin': {...}, 'growth_rate': ..., 'X_Scale': ..., ...}, ...]


def create_efficiency_frontier_plots_base_only(configurations):
    """
    Create efficiency frontier plots for actuarial simulation results.
    
    FIXED VERSION: Includes comprehensive error handling for pcolormesh issues.

    Parameters:
    -----------
    configurations : list of dict
        List of configuration dictionaries with simulation results

    Output:
    -------
    Saves PNG files for each X_Scale with efficiency frontier plots.
    """

    # Get unique values for separate charts
    x_scales = sorted(set(c['X_Scale'] for c in configurations))
    lr = sorted(set(c['LR'] for c in configurations))
    caps = sorted(set(c['Cap'] for c in configurations))

    # Define percentiles for overlay
    highlight_percentiles = [0.50, 0.75, 0.90, 0.95, 0.99]
    legend_percentiles = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]
    all_percentiles = np.arange(0.01, 1.00, 0.01).tolist()

    for cap in caps:
        # Create a plot for each X_Scale
        for scale in x_scales:
            for loss_ratio in lr:
                if scale != 1.0 or loss_ratio != 0.6:
                    continue

                config_subset = [c for c in configurations if c['Cap'] == cap and c['X_Scale'] == scale and c['LR'] == loss_ratio]
                # Get unique values for faceting
                x_shapes = sorted(set(float(c['X_Shape']) for c in config_subset))
                x_shapes = [s for s in x_shapes if s != 0] # Filter out 0
                base_shape = 1.0
                x_shapes = [base_shape]
                x_thresholds = sorted({(c['X_Th_%le'] if isinstance(c['X_Th_%le'], (float, np.floating)) else 0.0) for c in config_subset})
                x_thresholds = [s for s in x_thresholds if s != 0] # Filter out 0
                base_threshold = 0.001
                x_thresholds = [base_threshold]

                # Filter configurations for this scale
                scale_configs = config_subset

                # Create figure with subplots
                n_rows = len(x_thresholds)
                n_cols = len(x_shapes)

                fig, axes = plt.subplots(n_rows, n_cols,
                                        figsize=(5*3, 4*2),
                                        dpi=300,
                                        squeeze=False)

                # Set overall title
                scale_label = f"{float(scale):.1f}x" if scale != '1' else "Baseline"
                lr_pct = f"{loss_ratio*100:.0f}%"
                threshold_pct = f"{base_threshold*100:.2f}%"
                fig.suptitle(f'Efficiency Frontier (Init Cap: {cap/1_000_000:.0f}M; Scale: {scale_label}; Loss Ratio: {lr_pct}; Tail Threshold: {threshold_pct}; Shape: {base_shape})',
                            fontsize=16, fontweight='bold', y=0.995)

                # Create colormap for percentiles
                cmap = plt.cm.viridis

                # Plot each facet
                for i, threshold in enumerate(x_thresholds):
                    for j, shape_param in enumerate(x_shapes):
                        ax = axes[i, j]

                        # Debug: Check what values we're comparing
                        if threshold == 0 or shape_param == 0:
                            print(
                                f"\n=== Debug for threshold={threshold}, shape_param={shape_param} ===")
                            print(
                                f"shape_param type: {type(shape_param)}, value: {repr(shape_param)}")
                            print(
                                f"threshold type: {type(threshold)}, value: {repr(threshold)}")

                            # Sample a few configs to see their actual values
                            for c in scale_configs[:3]:
                                print(
                                    f"  X_Shape: {repr(c['X_Shape'])} (type: {type(c['X_Shape'])})")
                                print(
                                    f"  X_Th_%le: {repr(c['X_Th_%le'])} (type: {type(c['X_Th_%le'])})")
                                print(
                                    f"  float(c['X_Shape']) == shape_param: {float(c['X_Shape']) == shape_param}")
                                print(
                                    f"  Threshold match: {c['X_Th_%le'] == threshold or (c['X_Th_%le'] is None and threshold == 0)}")
                                print()

                        # Filter for this facet
                        facet_configs = [c for c in scale_configs
                                        if (c['X_Th_%le'] == threshold or (c['X_Th_%le'] == 'None' and threshold == 0))
                                        and float(c['X_Shape']) == shape_param]

                        if (threshold == 0 or shape_param == 0):
                            print(len(facet_configs))

                        # Intermediate pass - Create smooth gradient using pcolormesh
                        # Organize data by percentile
                        percentile_data = {pct: [] for pct in all_percentiles}

                        for config in facet_configs:
                            # Calculate log of policy limit for x-axis
                            if config.get('NOINS', False):
                                continue  # Skip NOINS
                            else:
                                pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                    config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                                x_value = np.log10(float(pol_lim))
                            
                            # Collect percentile values
                            if 'growth_rate_ci' in config and config['growth_rate_ci']:
                                for pct in all_percentiles:
                                    pct_key = f"{pct:.2f}" if f"{pct:.2f}" in config['growth_rate_ci'] else pct
                                    if pct_key in config['growth_rate_ci']:
                                        growth = config['growth_rate_ci'][pct_key]
                                        percentile_data[pct].append((x_value, growth, float(pol_lim)))

                        # Get unique x values (policy limits) and sort
                        unique_x_values = sorted(set(p[0] for pct_list in percentile_data.values() for p in pct_list))

                        if len(unique_x_values) > 1:
                            # Create interpolation grid with many points for smooth gradient
                            x_interp = np.linspace(min(unique_x_values), max(unique_x_values), 200)
                            
                            # For each percentile, interpolate growth rates across policy limits
                            from scipy.interpolate import interp1d
                            
                            y_grid = []
                            valid_percentiles = []
                            
                            for pct in sorted(all_percentiles):
                                if len(percentile_data[pct]) > 1:
                                    # Sort by x value
                                    sorted_data = sorted(percentile_data[pct], key=lambda p: p[0])
                                    x_data = np.array([p[0] for p in sorted_data])
                                    y_data = np.array([p[1] for p in sorted_data])
                                    
                                    # CRITICAL FIX: Remove non-finite values and use bounded extrapolation
                                    valid_mask = np.isfinite(x_data) & np.isfinite(y_data)
                                    if np.sum(valid_mask) >= 2:  # Need at least 2 points
                                        x_clean = x_data[valid_mask]
                                        y_clean = y_data[valid_mask]
                                        
                                        try:
                                            # Use bounded fill values instead of 'extrapolate' to prevent inf
                                            f = interp1d(x_clean, y_clean, kind='linear', 
                                                        bounds_error=False,
                                                        fill_value=(y_clean[0], y_clean[-1]))
                                            y_interp = f(x_interp)
                                            
                                            # Validate interpolation output
                                            if np.all(np.isfinite(y_interp)):
                                                y_grid.append(y_interp)
                                                valid_percentiles.append(pct)
                                            else:
                                                print(f"Warning: Non-finite interpolation at pct={pct}")
                                        except Exception as e:
                                            print(f"Interpolation failed at pct={pct}: {e}")
                            
                            if len(y_grid) > 1:
                                # Convert to 2D array: rows = percentiles, columns = x positions
                                y_grid = np.asarray(y_grid, dtype=np.float64)
                                
                                # CRITICAL FIX: Check for MaskedArray and validate data
                                if isinstance(y_grid, np.ma.MaskedArray):
                                    print(f"Warning: y_grid is MaskedArray, converting...")
                                    y_grid = y_grid.filled(np.nan)
                                
                                # Final validation: ensure all finite values
                                if not np.all(np.isfinite(y_grid)):
                                    print(f"Warning: Non-finite values in y_grid, cleaning...")
                                    # Replace inf with max/min finite
                                    finite_mask = np.isfinite(y_grid)
                                    if np.any(finite_mask):
                                        max_finite = np.max(y_grid[finite_mask])
                                        min_finite = np.min(y_grid[finite_mask])
                                        y_grid = np.where(np.isposinf(y_grid), max_finite, y_grid)
                                        y_grid = np.where(np.isneginf(y_grid), min_finite, y_grid)
                                        # Fill NaN with column means
                                        for col_idx in range(y_grid.shape[1]):
                                            col = y_grid[:, col_idx]
                                            if np.any(np.isnan(col)):
                                                col_mean = np.nanmean(col)
                                                y_grid[np.isnan(col), col_idx] = col_mean
                                
                                # Only proceed if we have valid data
                                if np.all(np.isfinite(y_grid)):
                                    # Create meshgrid for pcolormesh
                                    X, Y = np.meshgrid(x_interp, valid_percentiles)
                                    
                                    # Ensure meshgrids are regular arrays
                                    X = np.asarray(X, dtype=np.float64)
                                    Y = np.asarray(Y, dtype=np.float64)
                                    
                                    # Use pcolormesh with the colormap
                                    mesh = ax.pcolormesh(X, y_grid, Y, 
                                                        cmap=cmap, 
                                                        shading='gouraud',  # Smooth interpolation
                                                        vmin=0, vmax=1,
                                                        zorder=5)
                                else:
                                    print(f"Skipping pcolormesh - data still contains non-finite values")

                        # Second pass: Overlay specific percentiles with shapes
                        for config in facet_configs:
                            # Get risk of ruin at year 25
                            # ror_25 = config['risk_of_ruin'].get(25, config['risk_of_ruin'].get('25'))

                            # Calculate log of policy limit for x-axis
                            if config.get('NOINS', False):
                                x_value = 0  # or np.nan if you want to exclude NOINS from plot
                            else:
                                pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                    config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                                x_value = np.log10(float(pol_lim))

                            # Plot mean in #cc0000
                            if 'growth_rate' in config:
                                growth_mean = config['growth_rate']
                                ax.scatter(x_value, growth_mean,
                                            marker='o',
                                            c='#cc0000',
                                            alpha=1.0,
                                            edgecolors='#cc0000',
                                            linewidths=1.5,
                                            zorder=10)


                        ### Draw lines connecting means across limits #######
                        # Collect (x, growth_rate, raw_limit) tuples to connect mean growth rates
                        mean_points = []
                        
                        for config in facet_configs:
                            # Skip NOINS for connecting line (only plot actual policy limits)
                            if config.get('NOINS', False):
                                continue
                            pol_lim = config.get('Pol_Lim', [''])[0] if isinstance(
                                config.get('Pol_Lim'), list) else config.get('Pol_Lim', '')
                            try:
                                x_value = np.log10(float(pol_lim))
                            except (TypeError, ValueError):
                                continue
                            if 'growth_rate' in config and config['growth_rate'] is not None:
                                mean_points.append((x_value, float(config['growth_rate']), float(pol_lim)))
                        
                        if len(mean_points) > 1:
                            # Sort by actual (non-log) policy limit to ensure correct ordering
                            sorted_points = sorted(mean_points, key=lambda p: p[2])
                            x_coords = [p[0] for p in sorted_points]
                            y_coords = [p[1] for p in sorted_points]
                            ax.plot(x_coords, y_coords,
                                    color='#cc0000',
                                    linewidth=2,
                                    alpha=1.0,
                                    zorder=9)

                        # Formatting
                        ax.set_xlabel('Policy Limit (log scale)', fontsize=10)
                        ax.set_ylabel('Growth Rate', fontsize=10)
                        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

                        # Set title for each subplot
                        # thresh_label = threshold if threshold != 'None' else 'No Tail'
                        # ax.set_title(f'Threshold: {thresh_label}\nShape: {shape_param}',
                        #             fontsize=9)

                        # Set axis limits
                        # ax.set_xlim(x_min, x_max)
                        ax.set_xlim(np.log10(25000000* 0.9),
                                    np.log10(500000000 * 1.1))
                        # ax.set_ylim(y_min, y_max)

                        # Set x-ticks to show actual policy limit values
                        ax.set_xticks([np.log10(25e6),
                                        np.log10(50e6),
                                        np.log10(75e6),
                                        np.log10(100e6), 
                                        np.log10(150e6),
                                        np.log10(250e6),
                                        np.log10(350e6),
                                        np.log10(500e6)])
                        ax.set_xticklabels(['$25M',
                                            '$50M',
                                            '$75M', 
                                            '$100M', 
                                            '$150M',
                                            '$250M', 
                                            '$350M',
                                            '$500M'])

                # Color legend (percentiles + mean)
                color_elements = []
                for pct in legend_percentiles:
                    color_elements.append(mpatches.Patch(color=cmap(pct),
                                                        label=f'{int(pct*100)}th %ile',
                                                        edgecolor='black',
                                                        linewidth=0.5))
                color_elements.append(mpatches.Patch(color='#cc0000',
                                                    label='Mean',
                                                    edgecolor='#cc0000',
                                                    linewidth=0.5))

                fig.legend(handles=color_elements, loc='upper center',
                            bbox_to_anchor=(0.5, 0.0), title='Percentile',
                            frameon=False, fontsize=9, ncol=len(color_elements))

                plt.tight_layout(rect=[0, 0, 1, 0.99])

                # Save figure
                filename = f'cache/efficiency_frontier_base_cap({cap/1_000_000:.0f}M).png'
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                print(f'Saved: {filename}')
                plt.close()


create_efficiency_frontier_plots_base_only(ins_only_vals)

C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\869043613.py:309: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color=cmap(pct),
C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\869043613.py:313: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color='#cc0000',


Saved: cache/efficiency_frontier_base_cap(25M).png


C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\869043613.py:309: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color=cmap(pct),
C:\Users\alexf\AppData\Local\Temp\ipykernel_35220\869043613.py:313: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  color_elements.append(mpatches.Patch(color='#cc0000',


Saved: cache/efficiency_frontier_base_cap(50M).png
